In [23]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse
import cvxpy as cp
import relplot as rp
import math

In [24]:
# produce S_0, the simpliest constraints for lipschitz
def smCE_LP(S):
    (x_list, y_list) = S
    indices = np.argsort(x_list)
    x_list = x_list[indices]
    y_list = y_list[indices]
    A = np.diag([1]*len(x_list))
    A = (A -np.roll(A, 1, axis = 1))[0:len(x_list)-1]
    A = np.concatenate([A, -A])
    b = (x_list- np.roll(x_list,1,axis=0))[1:len(x_list)]
    b = np.concatenate([b,b])
    c = y_list - x_list
    return c, A, b

In [25]:
# produce S_base constraints in paper 
def smCE_uc(S):
    (x_list, y_list) = S
    n = len(x_list)
    indices = np.argsort(x_list)
    x_list = x_list[indices]
    y_list = y_list[indices]
    A = np.diag([1]*len(x_list))
    A = (A -np.roll(A, 1, axis = 1))[0:len(x_list)-1]
    temp = np.concatenate([A, -A])
    counter = temp.shape[0]
    A = np.zeros((counter+ 2*2**int(np.log(n)/np.log(2))-2, n))
    A[0:counter] = temp
    k = int(np.log2(n-1))
    for i in range(1, k+1):
        for j in range(1, 2**(k - i)+1):
            A[counter][(j-1)*2**i] = 1
            A[counter][j*2**i] = -1

            A[counter+1][(j-1)*2**i] = -1         
            A[counter+1][j*2**i] = 1
            counter +=2
    b = np.abs(A@x_list)
    c = y_list - x_list
    return c, A, b

In [26]:
def prepare_dataset(n, skew_function, e):
    fa = []
    ya = []
    for _ in range(n):
        f = np.random.uniform(0,1-e)
        y = int(np.random.uniform() > 1 - skew_function(f))
        fa.append(f)
        ya.append(y)
    return (np.array(fa), np.array(ya))

In [27]:
S_calibrated = prepare_dataset(2**6+1, lambda x:x+0.1, 0.1)
c,A,b = smCE_uc(S_calibrated)
n = len(S_calibrated[0])
x = cp.Variable(n)
objective = cp.Minimize(np.array([c/n]) @ x +2*math.log(n)*cp.norm(cp.maximum(A @ x-b, np.zeros(len(A)) ),"inf"))
constraints = [-1 <= x, x <= 1]
prob = cp.Problem(objective, constraints)
result = -prob.solve()
print(result)

0.11340232597018096


In [28]:
# %%
def ATmul(x, part = False):
    n = len(x)
    if part:
        ret = np.zeros(2*(n-1) + 1) 
    else:
        ret = np.zeros(2*(n-1) + 2*2**int(np.log2(n))-1) #should be -2 but -1 for last row with all 0
    ret[0:(n-1)] = x[:-1] - x[1:]
    ret[n-1:2*(n-1)] = -x[:-1] + x[1:]
    if part:
        return ret/2
    k = int(np.log2(n-1))
    counter = 2*(n-1)
    for i in range(1, k+1):
        indices = np.array(range(2**(k - i)+1))
        indices = 2**i * indices
        ret[counter: counter + (2**(k - i))] = (x[indices])[:-1] - (x[indices])[1:]
        ret[counter+ (2**(k - i)): counter + 2*(2**(k - i))] = (-x[indices])[:-1] + (x[indices])[1:]     
        counter += 2 * (2**(k - i))
    return ret/2

In [29]:
def A_absTmul(x, part = False):
    n = len(x)
    if part:
        ret = np.zeros(2*(n-1) + 1) 
    else:
        ret = np.zeros(2*(n-1) + 2*2**int(np.log2(n))-1) #should be -2 but -1 for last row with all 0
    ret[0:(n-1)] = x[:-1] + x[1:]
    ret[n-1:2*(n-1)] = x[:-1] + x[1:]
    if part:
        return ret/2
    k = int(np.log2(n-1))
    counter = 2*(n-1)
    for i in range(1, k+1):
        indices = np.array(range(2**(k - i)+1))
        indices = 2**i * indices
        ret[counter: counter + (2**(k - i))] = (x[indices])[:-1] + (x[indices])[1:]
        ret[counter+ (2**(k - i)): counter + 2*(2**(k - i))] = (x[indices])[:-1] + (x[indices])[1:]     
        counter += 2 * (2**(k - i))
    return ret/2

In [30]:
def Amul(y, size, part = False):
    n = len(y)
    ret = np.zeros(size) #should be -2 but -1 for last row with all 0
    counter = 0
    k = int(np.log2(size-1))
    for i in range(k):
        ret[0] += y[counter] - y[counter+ (2**(k - i))]
        ret[-1] += -y[counter+ (2**(k - i))-1] + y[counter+ 2*(2**(k - i))-1]
        indices = np.array(range(1, 2**(k - i)))
        indices = 2**i * indices
        ret[indices] += -(y[counter: counter + 2**(k - i)])[:-1] + (y[counter: counter + 2**(k - i)])[1:]
        counter +=(2**(k - i))
        ret[indices] += (y[counter: counter + 2**(k - i)])[:-1] - (y[counter: counter + 2**(k - i)])[1:]      
        counter +=(2**(k - i))
        if part:
            return ret/2
    ret[0] += y[counter] - y[counter+1]
    ret[-1] += -y[counter] + y[counter+1] 
    return ret/2

In [31]:
def A_absmul(y, size, part = False):
    n = len(y)
    ret = np.zeros(size) #should be -2 but -1 for last row with all 0
    counter = 0
    k = int(np.log2(size-1))
    for i in range(k):
        ret[0] += y[counter] + y[counter+ (2**(k - i))]
        ret[-1] += y[counter+ (2**(k - i))-1] + y[counter+ 2*(2**(k - i))-1]
        indices = np.array(range(1, 2**(k - i)))
        indices = 2**i * indices
        ret[indices] += (y[counter: counter + 2**(k - i)])[:-1] + (y[counter: counter + 2**(k - i)])[1:]
        counter +=(2**(k - i))
        ret[indices] += (y[counter: counter + 2**(k - i)])[:-1] + (y[counter: counter + 2**(k - i)])[1:]      
        counter +=(2**(k - i))
        if part:
            return ret/2
    ret[0] += y[counter] + y[counter+1]
    ret[-1] += y[counter] + y[counter+1] 
    return ret/2

In [32]:
# Our box simplex implementation with p = True for S_base and p = False for S_0
def box_simplex(size, b, c, epsilon, p = False, f = None):
    n = size[0]
    L = 2
    x = np.zeros(size[0])
    y = np.ones(size[1]) / size[1]
    y_bar = np.ones(size[1]) / size[1]
    x_hat = np.zeros(size[0])
    y_hat = np.zeros(size[1]) 
    T = int( 48*np.log(size[1]) * L / epsilon)
    
    #A = A / L
    c = c / (L*np.log2(n))
    #A_abs = np.abs(A)
    
    for t in range(T):
        # Gradient oracle start
        #g_x = (A @ y + c)/3
        g_x = (Amul(y, n,part = p) + c)/3
        #g_y = (b - A.T @ x)/3 
        g_y = (b - ATmul(x,part = p))/3 
        
        # Step 7
        x_star = np.clip( -(g_x - 2 * x * A_absmul(y, n, part = p)) / (2 * A_absmul(y,n,part = p)+0.000001), -1, 1)
        
        # Step 8
        #y_p = y * np.exp((-1 / 2) * (g_y + A_abs.T @ (x_star**2) - A_abs.T @ (x**2)))
        y_p = y * np.exp((-1 / 2) * (g_y + A_absTmul(x_star**2,part = p) - A_absTmul(x**2,part = p)))
        y_p = y_p / y_p.sum()  # Ensure y_star is a probability vector
        
        # Step 9
        #x_p = np.clip( - (g_x - 2 * np.diag(x)@ A_abs @ y) / (2 * A_abs @ y_p+0.000001), -1, 1)
        x_p = np.clip( - (g_x - 2 * x* A_absmul(y, n,part = p)) / (2 * A_absmul(y_p, n,part = p)+0.000001), -1, 1)
        
        # Step 10: Running average maintenance
        x_hat = x_hat  + x_p / T
        y_hat = y_hat + y_p / T
        
        # Step 12: Extrapolation oracle start
        #g_x = (A @ y_p + c)/6
        #g_y = (b-A.T @ x_p)/6
        g_x = (Amul(y_p, n,part = p) + c)/6
        g_y = (b-ATmul(x_p,part = p))/6
        
        # Step 14
        x_bar = np.clip( - (g_x - 2 * x * A_absmul(y,n,part = p)) / (2 * A_absmul(y_bar, n,part = p)+0.000001), -1, 1)
        
        # Step 15
        previous_y = y
        y = y_bar * np.exp((-1 / 2) * (g_y + A_absTmul(x_bar**2,part = p) + 2*(np.log(y_bar) - np.log(y)) - A_absTmul(x**2,part = p)))
        y = y / y.sum()  # Ensure y is a probability vector
        
        # Step 16
        x = np.clip( - (g_x - 2 * x * A_absmul(previous_y, n,part = p)) / (2 * A_absmul(y,n,part = p)+0.000001), -1, 1)
        y_bar = y * np.exp((-1 / 2) * (g_y + A_absTmul(x**2,part = p) + 2*(np.log(y) - np.log(previous_y)) - A_absTmul(x**2,part = p)))
        y_bar = y_bar / y_bar.sum()  # Ensure y is a probability vector    
    return x_hat, y_hat

In [33]:
(x_list, y_list) = S_calibrated
n = len(x_list)
indices = np.argsort(x_list)
x_list = x_list[indices]
y_list = y_list[indices]
b_p = np.abs(ATmul(x_list))
c_ = (y_list - x_list)/n

b_p = np.abs(ATmul(x_list, part = True))
x, y = box_simplex((n, 2*n-1), b_p, c_, 1/(math.log2(n)*n**0.5), p = True, f = None)
print(- c_@x)

0.10915796143662235
